In [11]:
# libraries

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import ast
from sklearn.model_selection import train_test_split


In [7]:
df = pd.read_csv("data/large_board_dataset2.csv")
#df2 = pd.read_csv("data/large_board_dataset2-mikala.csv")
#df = pd.concat([df1, df2], axis =0)


In [6]:
df2

,x,y,whose_turn,opponent_level,player_level,game,random_moves
0,"(np.float64(0.0), np.float64(0.0), np.float64(...",3,red,1500,1500,0,3
1,"(np.float64(0.0), np.float64(0.0), np.float64(...",4,yellow,1500,1500,0,3
2,"(np.float64(0.0), np.float64(0.0), np.float64(...",3,red,1500,1500,0,3
3,"(np.float64(0.0), np.float64(0.0), np.float64(...",3,yellow,1500,1500,0,3
4,"(np.float64(0.0), np.float64(0.0), np.float64(...",3,red,1500,1500,0,3
...,...,...,...,...,...,...,...
64587,"(np.float64(0.0), np.float64(0.0), np.float64(...",3,yellow,1500,1500,2999,10
64588,"(np.float64(0.0), np.float64(0.0), np.float64(...",0,red,1500,1500,2999,10
64589,"(np.float64(0.0), np.float64(0.0), np.float64(...",3,yellow,1500,1500,2999,10
64590,"(np.float64(0.0), np.float64(0.0), np.float64(...",2,red,1500,1500,2999,10


In [5]:
df.tail()

,x,y,whose_turn,opponent_level,player_level,game,random_moves
64587,"(np.float64(0.0), np.float64(0.0), np.float64(...",3,yellow,1500,1500,2999,10
64588,"(np.float64(0.0), np.float64(0.0), np.float64(...",0,red,1500,1500,2999,10
64589,"(np.float64(0.0), np.float64(0.0), np.float64(...",3,yellow,1500,1500,2999,10
64590,"(np.float64(0.0), np.float64(0.0), np.float64(...",2,red,1500,1500,2999,10
64591,"(np.float64(0.0), np.float64(0.0), np.float64(...",3,yellow,1500,1500,2999,10


In [8]:
df.head()
df["x"] = df["x"].apply(lambda x: np.array(ast.literal_eval(x)).reshape(2, 6, 7))


In [12]:
x = df["x"]
y = df["y"]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 22)

X_train = np.stack(X_train)
X_test = np.stack(X_test)
y_train = np.array(y_train)  
y_test = np.array(y_test)

X_train = X_train.transpose(0, 2, 3, 1)  # Convert (num_samples, 2, 6, 7) to (num_samples, 6, 7, 2)
X_test = X_test.transpose(0, 2, 3, 1)


In [13]:
def preprocess_board(board):
    if board["whose_turn"] != "red":
        # Flip the layers to make each board look like it's from the perspective of "red", aka plus
        board["x"] = board["x"][::-1, :, :]
    return board

In [40]:
df = df.apply(preprocess_board, axis=1)

In [34]:
cnn = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=64,kernel_size=(4,4),activation=tf.nn.relu,input_shape=(6, 7, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation=tf.nn.relu),
        # had to adjust data_format because it was treating the images as 2x6 with 7 channels
        tf.keras.layers.MaxPooling2D(pool_size = (2,2),strides=(1, 1), padding = "same"),
        tf.keras.layers.Conv2D(64, (1, 1), activation=tf.nn.relu),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation=tf.nn.relu,kernel_regularizer = tf.keras.regularizers.l2(0.0002)),
        tf.keras.layers.Dense(64,activation=tf.nn.relu,kernel_regularizer = tf.keras.regularizers.l2(0.0005)),
        tf.keras.layers.Dense(7,activation=tf.nn.softmax)
        ])

In [35]:
cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

cnn.summary()

cnn.fit(X_train,y_train,epochs=200,validation_split=0.2,batch_size=100)

pred_probs = cnn.predict(X_test)
pred = np.argmax(pred_probs, axis=1)
print(np.mean(pred==y_test))

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_32 (Conv2D)              │ (None, 3, 4, 64)       │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 1, 2, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 1, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 1, 2, 64)       │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_14 (Flatten)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,447 (427.53 KB)

 Trainable params: 109,447 (427.53 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
418/418 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.2369 - loss: 1.8818 - val_accuracy: 0.3861 - val_loss: 1.5725
Epoch 2/200
418/418 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.4162 - loss: 1.4974 - val_accuracy: 0.4485 - val_loss: 1.4256
Epoch 3/200
418/418 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.4753 - loss: 1.3618 - val_accuracy: 0.4671 - val_loss: 1.3798
Epoch 4/200
418/418 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.4927 - loss: 1.3102 - val_accuracy: 0.4710 - val_loss: 1.3688
Epoch 5/200
418/418 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5163 - loss: 1.2599 - val_accuracy: 0.4755 - val_loss: 1.3479
Epoch 6/200
418/418 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5347 - loss: 1.2185 - val_accuracy: 0.4827 - val_loss: 1.3487
Epoch 7/200
418/418 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5520 - loss: 1.1727 - val_accuracy: 0.4820 - val_loss: 1.3464
Epoch 8/200
418/418 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.5700 - loss: 1.1362 - val_ac

In [49]:
print(f"X_train dtype: {X_train.dtype}, shape: {X_train.shape}")


X_train dtype: object, shape: (52238,)


In [56]:
len(df)

65298